In [1]:
import numpy as np

def generate_log2system(int_length, fraction_length):
  #system 2^(I.F)
  print ("generating table for 2^(I.F), I=%d, F=%d "%(int_length,fraction_length))
  step  =  2**(-fraction_length)
  power_table = np.arange(-2**int_length,  2**int_length, step)
  print ("power 2 table: ", list(power_table))
  table = list(map(lambda x: 2.0**x, power_table))
  print ("system 2^(%d.%d): "%(int_length,fraction_length), table)
  return table

weight_table = generate_log2system(2,1)


generating table for 2^(I.F), I=2, F=1 
power 2 table:  [-4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]
system 2^(2.1):  [0.0625, 0.08838834764831845, 0.125, 0.1767766952966369, 0.25, 0.3535533905932738, 0.5, 0.7071067811865476, 1.0, 1.4142135623730951, 2.0, 2.8284271247461903, 4.0, 5.656854249492381, 8.0, 11.313708498984761]


In [2]:
import os
!nvidia-smi
!pip install transformers datasets
!pip install nlp

!git clone https://github.com/minhhn2910/QPyTorch
try:
  os.chdir('/content/QPyTorch')
except:
  pass
!git checkout posit-constant-generation

!pip install ./
!pip install ninja

try:
  os.chdir('/content/')
except:
  pass

import torch

if not torch.cuda.is_available():
  raise RuntimeError('Cannot run this cell without GPU runtime.')

import torch
import torch.nn as nn
import numpy as np




Mon Nov 18 22:29:00 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 26%   35C    P5    20W / 250W |     18MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
device = 'cuda'
model_id = 'gpt2-large'
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
# Print all layer names
print("Listing all layers in the model:")
for name, module in model.named_modules():
    print(name, ":", type(module))

Listing all layers in the model:
 : <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
transformer : <class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
transformer.wte : <class 'torch.nn.modules.sparse.Embedding'>
transformer.wpe : <class 'torch.nn.modules.sparse.Embedding'>
transformer.drop : <class 'torch.nn.modules.dropout.Dropout'>
transformer.h : <class 'torch.nn.modules.container.ModuleList'>
transformer.h.0 : <class 'transformers.models.gpt2.modeling_gpt2.GPT2Block'>
transformer.h.0.ln_1 : <class 'torch.nn.modules.normalization.LayerNorm'>
transformer.h.0.attn : <class 'transformers.models.gpt2.modeling_gpt2.GPT2SdpaAttention'>
transformer.h.0.attn.c_attn : <class 'transformers.pytorch_utils.Conv1D'>
transformer.h.0.attn.c_proj : <class 'transformers.pytorch_utils.Conv1D'>
transformer.h.0.attn.attn_dropout : <class 'torch.nn.modules.dropout.Dropout'>
transformer.h.0.attn.resid_dropout : <class 'torch.nn.modules.dropout.Dropout'>
transformer.h.0.ln_2 : <clas

In [ ]:
from transformers import pipeline
from transformers import set_seed
from qtorch_plus.quant import posit_quantize, float_quantize, configurable_table_quantize
import transformers
transformers.logging.set_verbosity_error()



#final with 3 layers skipped :
full_table = np.array([6.59179688e-03, 2.19726562e-02, 4.02832031e-02, 6.25000000e-02,
                8.51440430e-02, 2.10937500e-01, 1.21093750e-01, 5.36407471e-01,
                1.56250000e-02, 6.25000000e-02, 1.25000000e-01, 1.64062500e-01,
                2.92968750e-01, 6.25000000e-01, 8.43750000e-01, 4.37500000e-01,
                1.00000000e+00, 2.11486816e+00, 1.50000000e+00, 1.25000000e+00,
                4.25000000e+00, 3.08789062e+00, 8.25000000e+00, 6.00000000e+00])

weight_table = full_table[:8]
act_table = full_table[8:]
weight_table = np.sort(weight_table)
act_table = np.sort(act_table)
print (len(weight_table))
print (weight_table)
print (act_table)

def linear_weight(input):
    # return input
    # return configurable_table_quantize(input, torch.tensor(weight_table,dtype = torch.float), scale= 1.0)
    return posit_quantize(input,nsize=8, es=1, scale = 1)
    # return float_quantize(input,exp=4, man=3, rounding="nearest")

def linear_activation(input):
    # return input
    # return configurable_table_quantize(input,torch.tensor(act_table, dtype=torch.float), scale= 1.0)
    return posit_quantize(input,nsize=6, es=0, scale = 1)
    # return float_quantize(input,exp=4, man=3, rounding="nearest")

def forward_pre_hook_linear(m, input):
    return (linear_activation(input[0]),)

# model = model.to(device)
# layer_count = 0
# linear_layer_count = 0
# op_count = 0
# epsilon = 1e-12  # To avoid log(0)
# import transformers.modeling_utils as  modeling_utils

# for name, module in model.named_modules():
#     # Check if the module is quantizable
#     if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear) or isinstance(module, modeling_utils.Conv1D):
#         layer_count += 1
        
#         # Apply different quantization configurations for specific layers
#         if name == "transformer.h.0.attn.c_attn":
#             print(f"Quantizing layer {name} with Posit<8,1>")
#             module.weight.data = posit_quantize(module.weight.data, nsize=8, es=1)
#         elif name == "transformer.h.1.attn.c_attn":
#             print(f"Quantizing layer {name} with Posit<16,2>")
#             module.weight.data = posit_quantize(module.weight.data, nsize=16, es=2)
#         elif name == "transformer.h.2.attn.c_attn":
#             print(f"Quantizing layer {name} with Float<4,3>")
#             module.weight.data = float_quantize(module.weight.data, exp=4, man=3, rounding="nearest")
#         else:
#             # Default quantization
#             print(f"Quantizing layer {name} with default Posit<8,1>")
#             module.weight.data = posit_quantize(module.weight.data, nsize=8, es=1)

#         # Register forward pre-hook for activations
#         module.register_forward_pre_hook(forward_pre_hook_linear)

#         # Count operations
#         if isinstance(module, modeling_utils.Conv1D):
#             op_count += module.weight.shape[0] * module.weight.shape[1]
#         else:
#             op_count += module.in_features * module.out_features
    
#     # Handle embeddings separately
#     elif isinstance(module, nn.Embedding):
#         print(f"Quantizing embedding layer {name} with Posit<8,1>")
#         module.weight.data = posit_quantize(module.weight.data, nsize=8, es=1)


model = model.to(device)
layer_count = 0
op_count = 0
epsilon = 1e-12  # To avoid log(0)
import transformers.modeling_utils as modeling_utils

for name, module in model.named_modules():
    # Check if the module is quantizable
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear) or isinstance(module, modeling_utils.Conv1D):
        layer_count += 1
        print(f"Processing layer: {name}")

        # Flatten the weights and compute log2 scale
        weights_flattened = module.weight.data.cpu().numpy().flatten()
        log2_weights = np.log2(np.abs(weights_flattened) + epsilon)
        counts, bins = np.histogram(log2_weights, bins=100)
        max_bin_index = np.argmax(counts)
        x_with_max_frequency = (bins[max_bin_index] + bins[max_bin_index + 1]) / 2  # Bin center
        print(f"x_with_max_frequency for {name}: {x_with_max_frequency:.2f}")
        scale = 2 ** (-x_with_max_frequency)
        quantized_weights = posit_quantize(torch.tensor(module.weight.data, dtype=torch.float32), nsize=6, es=0, scale=scale)
        module.weight.data = quantized_weights

        print(f"Quantized layer {name} with scale factor: {scale:.2e}")
        module.register_forward_pre_hook(forward_pre_hook_linear)


        # Count operations
        if isinstance(module, modeling_utils.Conv1D):
            op_count += module.weight.shape[0] * module.weight.shape[1]
        else:
            op_count += module.in_features * module.out_features

    # Handle embeddings separately
    elif isinstance(module, nn.Embedding):
        print(f"Processing embedding layer: {name}")

        # Flatten the weights and compute log2 scale
        weights_flattened = module.weight.data.cpu().numpy().flatten()
        log2_weights = np.log2(np.abs(weights_flattened) + epsilon)

        # Compute histogram and find the bin with the maximum frequency
        counts, bins = np.histogram(log2_weights, bins=100)
        max_bin_index = np.argmax(counts)
        x_with_max_frequency = (bins[max_bin_index] + bins[max_bin_index + 1]) / 2  # Bin center
        print(f"x_with_max_frequency for {name}: {x_with_max_frequency:.2f}")

        # Apply quantization with scale based on x_with_max_frequency
        scale = 2 ** (-x_with_max_frequency)
        quantized_weights = posit_quantize(torch.tensor(module.weight.data, dtype=torch.float32), nsize=6, es=0, scale=scale)
        module.weight.data = quantized_weights

        print(f"Quantized embedding layer {name} with scale factor: {scale:.2e}")

print("Total layers processed:", layer_count)
print("MAC operation count:", op_count)

print("MAC operation count:", op_count)
print("Layer count:", layer_count)

print ("MAC operation count ", op_count)
print ("Layer count ", layer_count)


# from nlp import load_dataset

from datasets import load_dataset
test = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')
#model = model.to(device)
def generate_text(model_new):
  text_generation = pipeline("text-generation", model=model_new, tokenizer=tokenizer, device = 0)
  #set_seed(42)
  prefix_texts = ["Machine learning is the study of",
                    "In the 19th century, the invention",
                    "A robot was created",
                    "One day I will"
                  ]
  for prefix_text in prefix_texts:
    #generated_text= text_generation(prefix_text, max_length=50, do_sample=False )[0]
    set_seed(42)
    generated_text= text_generation(prefix_text, max_length=50, num_return_sequences=3)
    print(generated_text[0]['generated_text'])
    print ("-----------------")
    print(generated_text[1]['generated_text'])
    print ("-----------------------------------")
    print(generated_text[2]['generated_text'])
    print ("-----------------------------------")
generate_text(model)

print ("\n FP32 ref : \n")
# model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
# generate_text(model)


Using /home/yg9bq/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Emitting ninja build file /home/yg9bq/.cache/torch_extensions/py310_cu118/quant_cpu/build.ninja...
Building extension module quant_cpu...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module quant_cpu...
Using /home/yg9bq/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/yg9bq/.cache/torch_extensions/py310_cu118/quant_cuda/build.ninja...
/home/yg9bq/miniconda3/envs/posit/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


ninja: no work to do.


Building extension module quant_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module quant_cuda...


8
[0.0065918  0.02197266 0.0402832  0.0625     0.08514404 0.12109375
 0.2109375  0.53640747]
[0.015625   0.0625     0.125      0.1640625  0.29296875 0.4375
 0.625      0.84375    1.         1.25       1.5        2.11486816
 3.08789062 4.25       6.         8.25      ]
Processing embedding layer: transformer.wte
x_with_max_frequency for transformer.wte: -4.26
Quantized embedding layer transformer.wte with scale factor: 1.91e+01
Processing embedding layer: transformer.wpe
x_with_max_frequency for transformer.wpe: -7.87
Quantized embedding layer transformer.wpe with scale factor: 2.33e+02
Processing layer: transformer.h.0.attn.c_attn
x_with_max_frequency for transformer.h.0.attn.c_attn: -4.47
Quantized layer transformer.h.0.attn.c_attn with scale factor: 2.22e+01
Processing layer: transformer.h.0.attn.c_proj
x_with_max_frequency for transformer.h.0.attn.c_proj: -5.24
Quantized layer transformer.h.0.attn.c_proj with scale factor: 3.78e+01
Processing layer: transformer.h.0.mlp.c_fc


/tmp/ipykernel_31802/440090267.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  quantized_weights = posit_quantize(torch.tensor(module.weight.data, dtype=torch.float32), nsize=5, es=1, scale=scale)
/tmp/ipykernel_31802/440090267.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  quantized_weights = posit_quantize(torch.tensor(module.weight.data, dtype=torch.float32), nsize=5, es=1, scale=scale)


x_with_max_frequency for transformer.h.0.mlp.c_fc: -4.50
Quantized layer transformer.h.0.mlp.c_fc with scale factor: 2.27e+01
Processing layer: transformer.h.0.mlp.c_proj
x_with_max_frequency for transformer.h.0.mlp.c_proj: -5.06
Quantized layer transformer.h.0.mlp.c_proj with scale factor: 3.34e+01
Processing layer: transformer.h.1.attn.c_attn
x_with_max_frequency for transformer.h.1.attn.c_attn: -4.60
Quantized layer transformer.h.1.attn.c_attn with scale factor: 2.42e+01
Processing layer: transformer.h.1.attn.c_proj
x_with_max_frequency for transformer.h.1.attn.c_proj: -5.01
Quantized layer transformer.h.1.attn.c_proj with scale factor: 3.22e+01
Processing layer: transformer.h.1.mlp.c_fc
x_with_max_frequency for transformer.h.1.mlp.c_fc: -4.44
Quantized layer transformer.h.1.mlp.c_fc with scale factor: 2.17e+01
Processing layer: transformer.h.1.mlp.c_proj
x_with_max_frequency for transformer.h.1.mlp.c_proj: -4.69
Quantized layer transformer.h.1.mlp.c_proj with scale factor: 2.58e+01

In [5]:
torch.cuda.empty_cache()

import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 1024

lls = []
input_size = encodings.input_ids.size(1)
print("Input size:", input_size)

for i in tqdm(range(0, input_size, stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, input_size)
    trg_len = end_loc - i

    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        log_likelihood = outputs[0] * trg_len
        # print(f"Log Likelihood for step {i}: {log_likelihood}")

    lls.append(log_likelihood)

if lls:  # Ensure lls is not empty
    ppl = torch.exp(torch.stack(lls).sum() / input_size)
    print("Perplexity:", ppl.item())
else:
    print("No log likelihoods calculated.")


Input size: 287644


100%|██████████| 281/281 [01:04<00:00,  4.36it/s]

Perplexity: 25.837797164916992
